In [1]:
# !pip install --upgrade scipy

In [1]:
'''
Authors: Satvik Dixit and Daniel M. Low
License: See license in github repository
'''

import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import soundfile as sf
import os 
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit
from sklearn import metrics

In [2]:
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

In [6]:


pd.set_option("display.max_columns", None)
# pd.options.display.width = 0


on_colab = False

if on_colab:
  from google.colab import drive
  project_name = 'project_name'
  drive.mount('/content/drive')
  input_dir = f'/content/drive/MyDrive/datum/{project_name}/data/input/'
  output_dir = f'/content/drive/MyDrive/datum/{project_name}/data/output/'
else:
  input_dir = './../../../../data/blake_private/audio/'
  output_dir = './../../../../data/blake_private/features/audio/cpp/'

# os.makedirs(output_dir, exist_ok=True)



In [8]:
audio_files = os.listdir(input_dir+'blake_16khz/')
audio_files.sort()

In [15]:
try: audio_files.remove('.DS_Store')
except: pass

In [9]:
# audio_files_speech = [n for n in audio_files if 'Speech_' in n]

# Feature extraction

In [10]:
# from importlib import reload
# reload(cpp)

In [11]:
import numpy as np
from scipy import signal
import io
import math
import numpy.matlib



# Hanning
def hanning(N):
    x = np.array([i/(N+1) for i in range(1, int(np.ceil(N/2))+1)])
    w = 0.5-0.5*np.cos(2*np.pi*x)
    w_rev = w[::-1]
    return np.concatenate((w, w_rev[int((np.ceil(N % 2))):]))



# function for computing cepstral peak prominence
def cpp(x, fs, normOpt = 'line', dBScaleOpt = True):
    """
    Computes cepstral peak prominence for a given signal 
    Parameters
    Python implementation by Satvik Dixit and Daniel Low (MIT)
    Based on MatLab implementation from John Kane kanejo@tcd.ie
    based on Hillenbrand, J., Houde, R. A. (1996) ``Acoustic correlates of
      breathy vocal quality: dysphonic voices and continuous
      speech'', Journal of Speech and Hearing research 39:311-321

    This function is part of the Covarep project: 
    http://covarep.github.io/covarep
    
    -----------
    x: ndarray
        The audio signal
    fs: integer
        The sampling frequency
    normOpt: string
        'line', 'mean' or 'nonorm' for selecting normalisation type
    dBScaleOpt: binary
        True or False for using decibel scale
    Returns
    -----------
    cpp: ndarray
        The CPP with time values 
    """
    # Settings
    frame_length = int(np.round_(0.04*fs))
    frame_shift = int(np.round_(0.01*fs))
    half_len = int(np.round_(frame_length/2))
    x_len = len(x)
    frame_len = half_len*2 + 1
    NFFT = 2**(math.ceil(np.log(frame_len)/np.log(2)))
    quef = np.linspace(0, frame_len/1000, NFFT)

    # Allowed quefrency range
    pitch_range = [60, 333.3]
    quef_lim = [int(np.round_(fs/pitch_range[1])),
                int(np.round_(fs/pitch_range[0]))]
    quef_seq = range(quef_lim[0]-1, quef_lim[1])

    # Time samples
    time_samples = np.array(
        range(frame_length+1, x_len-frame_length+1, frame_shift))
    N = len(time_samples)
    frame_start = time_samples-half_len
    frame_stop = time_samples+half_len

    # High-pass filtering
    HPfilt_b = [1 - 0.97]
    x = signal.lfilter(HPfilt_b, 1, x)

    # Frame matrix
    frameMat = np.zeros([NFFT, N])
    for n in range(0, N):
        frameMat[0: frame_len, n] = x[frame_start[n]-1:frame_stop[n]]


    win = hanning(frame_len)
    winmat = numpy.matlib.repmat(win, N, 1).transpose()
    frameMat = frameMat[0:frame_len, :]*winmat

    # Cepstrum
    SpecMat = np.abs(np.fft.fft(frameMat, axis=0))
    SpecdB = 20*np.log10(SpecMat)
    if dBScaleOpt:
        ceps = 20*np.log10(np.abs(np.fft.fft(SpecdB, axis=0)))
    else:
        ceps = 2*np.log(np.abs(np.fft.fft(SpecdB, axis=0)))

    # Finding the peak
    ceps_lim = ceps[quef_seq, :]
    ceps_max = ceps_lim.max(axis=0)
    max_index = ceps_lim.argmax(axis=0)

    # Normalisation
    ceps_norm = np.zeros([N])
    if normOpt == 'line':
        for n in range(0, N):
            p = np.polyfit(quef_seq, ceps_lim[:, n], 1)
            ceps_norm[n] = np.polyval(p, quef_seq[max_index[n]])
    elif normOpt == 'mean':
        ceps_norm = np.mean(ceps_lim)

    cpp = ceps_max-ceps_norm

    return cpp, time_samples

In [18]:
x, fs = sf.read(input_dir+'blake_16khz/'+audio_files[0])


In [20]:
%%time 
cpp_df = {}
did_not_work = []
for audio_file in audio_files:
    try:
        x, fs = sf.read(input_dir+'blake_16khz/'+audio_file)
        cpp_i, time_samples_i = cpp(x, fs, normOpt = 'line', dBScaleOpt = True)
        cpp_amean = np.mean(cpp_i) 
        cpp_stddevNorm = np.std(cpp_i) / np.mean(cpp_i)
        cpp_percentile20, cpp_percentile50,cpp_percentile80 = np.percentile(cpp_i, [20,50,80])
        cpp_df[audio_file] = [cpp_amean, cpp_stddevNorm, cpp_percentile20, cpp_percentile50,cpp_percentile80] 
    except:
        print(audio_file, 'did not work')
        did_not_work.append(audio_file)


/Users/danielmlow/miniconda3/envs/blake/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log10


CPU times: user 39min 16s, sys: 15.6 s, total: 39min 32s
Wall time: 40min 4s


In [21]:
cpp_df = pd.DataFrame(cpp_df).T.reset_index()
cpp_df.columns = ['filename', 'cpp_amean', 'cpp_stddevNorm', 'cpp_percentile20', 'cpp_percentile50','cpp_percentile80']
cpp_df = cpp_df.drop('cpp_percentile50', axis=1) # very similar to mean
cpp_df['filename'] = [n.replace('.wav', '') for n in cpp_df['filename'].values]
cpp_df.to_csv(output_dir+'cpp.csv')
cpp_df

,filename,cpp_amean,cpp_stddevNorm,cpp_percentile20,cpp_percentile80
0,00008C15-3259-40F4-A75E-7B0696B5FE04_442_1618_...,19.036997,0.251676,14.488487,23.757092
1,0016C554-7D5D-4600-96C4-2196F4416FC7_436_1620_...,16.846015,0.236657,13.600865,20.743847
2,001d57a9-9bf6-48ba-9f62-0ea0745cd3d1_1d03b12d-...,15.521122,0.164990,13.474102,17.492078
3,00200016-33AF-4C3F-AC50-A3E5880468F7_115_1265_...,14.272418,0.115142,12.870262,15.643025
4,0022169C-6220-4D62-8639-94B9E2C0DE06_2478_2226...,15.316328,0.176841,12.847559,17.662367
...,...,...,...,...,...
19662,ffd65cf1-4f02-4f69-8f02-1151bdb267a7_7c5de528-...,17.113881,0.274752,13.428778,21.798318
19663,ffe187f9-2d34-4b1d-966e-88d38b379f2c_9f02150d-...,NaN,NaN,NaN,NaN
19664,ffe3e2ad-e63f-458c-9040-b7e616bd68c3_b8643ad9-...,16.171865,0.227022,13.183138,19.469157
19665,ffe4c840-aa6d-4de1-ab8e-f4fd402532c6_24ca9764-...,16.492589,0.256005,13.212900,20.554531


In [22]:
cpp_df.to_csv(output_dir+'cpp.csv')

### Add to egemaps features DF

In [51]:
# egemaps features
egemaps_filenames = ['egemaps_vector_both.csv',
                    'egemaps_vector_speech.csv',
                    'egemaps_vector_vowel.csv'
                   ]

egemaps_features_df = {}
for i in egemaps_filenames:
    df_i = pd.read_csv(input_dir+'features/'+i, index_col = 0)
    egemaps_features_df[i]=df_i
    

In [54]:
for filename, df_i in egemaps_features_df.items():
    df_i_cpp = df_i.merge(cpp_df, on = ['filename'], how='inner')
    df_i_cpp.to_csv(input_dir+'features/'+filename.replace('.csv', '_cpp.csv'))
    
    

# Bootstrapping classification

In [13]:

models = [
    LogisticRegressionCV(solver='liblinear', penalty = 'l1', max_iter = 100),
    SGDClassifier(loss='log', penalty="elasticnet", early_stopping=True, max_iter = 5000),
    MLPClassifier(alpha = 1, max_iter= 1000),
    RandomForestClassifier(n_estimators= 100)
]


names = ['LogisticRegressionCV', 'SGDClassifier', "MLPClassifier","RandomForestClassifier"]

In [16]:
%%time

toy = False

for df_name, task_type_df in zip(['speech', 'vowel', 'both'], [
    'egemaps_vector_speech_cpp.csv',
    'egemaps_vector_vowel_cpp.csv', 
    'egemaps_vector_both_cpp.csv']):
    cpp_df = pd.read_csv(input_dir+f'features/{task_type_df}', index_col = 0)
    print(df_name, '\n====')

    for null_model in [True, False]:
        print('\npermute', null_model)
    
        if toy:
          n_bootstraps = 3
        else:
          n_bootstraps = 50

        # bs = cross_validation.Bootstrap(len(y), n_bootstraps=n_bootstraps, random_state=123,n_test = 0.2) # or add source code


        variables = ['cpp_amean', 'cpp_stddevNorm', 'cpp_percentile20', 'cpp_percentile80']
        X = cpp_df[variables].values
        y = cpp_df['target'].values
        if null_model:
            y = np.random.permutation(y) #CHECK
        groups = cpp_df['sid'].values

        y_pred_all = {}
        roc_auc_all = {}
        for model, name in zip(models, names):
          y_pred_all[name] = []
          roc_auc_all[name] = []
          pipe = Pipeline(steps=[
                  ('scaler', StandardScaler()), 
                  ('model', model)])


          ## Performing bootstrapping
          splitter = GroupShuffleSplit(n_splits=50, test_size=0.2, random_state=0)
          for i, (train_index, test_index) in enumerate(splitter.split(X, y, groups)):

              X_train, X_test = X[train_index], X[test_index]
              y_train, y_test = y[train_index], y[test_index]
              pipe.fit(X_train,y_train)     

              # # Evaluate
              # y_proba = pipe.predict_proba(X_test)       # Get predicted probabilities
              # y_proba_1 = y_proba[:,1]
              # y_pred = np.argmax(y_proba, axis=1) 
              # roc_auc = metrics.roc_auc_score(y_test, y_proba_1)  

              y_pred = pipe.predict(X_test) 
              roc_auc = metrics.roc_auc_score(y_test, y_pred)  # ROC AUC takes probabilities but here we match what pydra-ml does: https://github.com/nipype/pydra-ml/issues/56

              y_pred_all[name].append(y_pred)
              roc_auc_all[name].append(roc_auc)

        results_i = []
        for name in ['LogisticRegressionCV','MLPClassifier','RandomForestClassifier','SGDClassifier']:
          scores = roc_auc_all.get(name)
          roc_auc_median = np.round(np.median(scores),2)
          roc_auc_5 = np.round(np.percentile(scores, 5),2)
          roc_auc_95 = np.round(np.percentile(scores, 95),2)
          results_str = f'{roc_auc_median} ({roc_auc_5}–{roc_auc_95}; )'
          results_str = results_str.replace('0.', '.')
          results_i.append([name, results_str])
            
          if null_model:
            print(name, str(roc_auc_median).replace('0.', '.'))
        if not null_model:
            results_i_df = pd.DataFrame(results_i, ).T
            display(results_i_df)
            results_i_df.to_csv(output_dir+f'cpp/results_cpp_{df_name}_permute-{null_model}_{ts}.csv')

            
                




speech 
====

permute True
LogisticRegressionCV .5
MLPClassifier .46
RandomForestClassifier .55
SGDClassifier .5

permute False


,0,1,2,3
0,LogisticRegressionCV,MLPClassifier,RandomForestClassifier,SGDClassifier
1,.76 (.64–.84; ),.76 (.64–.84; ),.71 (.64–.78; ),.74 (.6–.84; )


vowel 
====

permute True


/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/_base.py:975: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/danielmlow/miniconda3/envs/pydra/lib/python3.7/site-packages/sklearn/svm/

LogisticRegressionCV .5
MLPClassifier .48
RandomForestClassifier .45
SGDClassifier .5

permute False


,0,1,2,3
0,LogisticRegressionCV,MLPClassifier,RandomForestClassifier,SGDClassifier
1,.82 (.73–.9; ),.82 (.71–.9; ),.77 (.65–.85; ),.77 (.4–.86; )


both 
====

permute True
LogisticRegressionCV .5
MLPClassifier .53
RandomForestClassifier .5
SGDClassifier .49

permute False


,0,1,2,3
0,LogisticRegressionCV,MLPClassifier,RandomForestClassifier,SGDClassifier
1,.72 (.65–.8; ),.74 (.68–.84; ),.72 (.65–.78; ),.68 (.44–.78; )


CPU times: user 3min 13s, sys: 2min 47s, total: 6min 1s
Wall time: 2min 48s
